# Ejercicio adicional de fin de semana: semana 2

Ahora usa todo lo que aprendiste en la semana 2 para construir un prototipo completo para la pregunta/respuesta técnica que creaste en el ejercicio de la semana 1.

Esto debería incluir una interfaz de usuario de Gradio, transmisión, uso del mensaje del sistema para agregar experiencia y la capacidad de cambiar entre modelos. ¡Puntos extra si puedes demostrar el uso de una herramienta!

Si te sientes audaz, ve si puedes agregar una entrada de audio para poder hablarle y hacer que responda con audio. ChatGPT o Claude pueden ayudarte, o envíame un correo electrónico si tienes preguntas.

Pronto publicaré una solución completa aquí, a menos que alguien se me adelante...

Hay tantas aplicaciones comerciales para esto, desde un tutor de idiomas hasta una solución de incorporación de empresas, pasando por una IA complementaria para un curso (¡como este!). No puedo esperar a ver tus resultados.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import requests

In [2]:
# Inicialización

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key sin configurar")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


# RAPID API - GOOGLE FLIGHT API

Documentación: https://rapidapi.com/DataCrawler/api/google-flights2

Rapidapi es un servicio de varias API y en este caso usaremo el de google flights para obtener los precios y los horarios.

Para ello es necesario obtener el "RAPIDAPI_KEY" y el "RAPIDAPI_HOST" el cual se lo consigue logeandote en rapid api, ademas de eso subscribiendote a la API que deseas usar (google flights en nuestro caso)
 
La API tiene varios endpoints para poder obnter aeropuertos, paises etc pero para este pequeño caso utilice el de "search flights" para poder mejorar nuestra herramienta. Pero sientete libre de crear mas tools con el resto de endpoints!!

RECOMENDACIÓN:

Recuerda que al subscribirte a la API de "google flights" tendra un total de llamadas gratuitas al mes, no los desperdicies, en caso de que se te termine, te puedes crear otra cuenta y volverte a subscribir o pagar, creo que son 10 o 15 llamadas máximo, asi que aprovechalos.

In [3]:
# Configuración de la API
rapid_api_key = os.getenv('RAPIDAPI_KEY')
rapid_api_host = os.getenv('RAPIDAPI_HOST')

if not rapid_api_key or not rapid_api_host:
    print("RapidAPI Key o RapidAPI Host no configurado")

In [ ]:
# Clase para manejar API de vuelos y retornar valores más exactos

class FlightAPI:
    
    def __init__(self, api_key, host):
        
        self.url = 'https://google-flights2.p.rapidapi.com/api/v1'
        self.headers = {
            "x-rapidapi-key": api_key,
            "x-rapidapi-host": host
        }
        
    def get_ticket_prices(self, origin, destination, outbound_date, inbound_date, travel_class='ECONOMY'):
        adults = 1 # Solo se considera un adulto
        currency = 'USD' # Se considera la moneda en dólares
        language_code = 'es' # Se considera el idioma en español
        url = f"{self.url}/searchFlights?departure_id={origin}&arrival_id={destination}&travel_class={travel_class}&adults={adults}&currency={currency}&language_code={language_code}&outbound_date={outbound_date}&inbound_date={inbound_date}"
        response = requests.get(url, headers=self.headers)
        data = response.json()
        itineraries = data.get('data', {}).get('itineraries', {})
        print(data)
        print(itineraries)
        return itineraries

In [5]:
# Instancia de la clase FlightAPI
flight_api = FlightAPI(rapid_api_key, rapid_api_host)

In [6]:
# OPENAI System
system_message = "Eres un asistente útil para una aerolínea llamada FlightAI. "
system_message += "Da respuestas breves y corteses, de no más de una oración. "
system_message += "Se siempre preciso. Si no sabes la respuesta, dilo."

In [7]:
# Función para obtener recomendaciones de vuelos
def get_tickets_suggests(origin, destination, outbound_date, inbound_date, travel_class='ECONOMY'):
    print(f"Buscando vuelos de {origin} a {destination} para el {outbound_date} al {inbound_date} en clase {travel_class}")
    itineraries = flight_api.get_ticket_prices(origin, destination, outbound_date, inbound_date, travel_class)
    return itineraries

In [8]:
msg_description = "Obtén información sobre los vuelos de un destino a otro entre dos fechas. Llámalo siempre que necesites saber el precio del billete o posibles vuelos."
msg_description += "Es obligatorio proporcionar el código IATA de la ciudad de origen y destino, al igual que fechas de salida y regreso."
msg_description += "En caso de que el usuario no te haya brindado la información necesaria, pidele que lo haga."
msg_description += "Si no puedes encontrar la información, dilo."
msg_description += """
Ejemplo:
¿Cuánto cuesta un billete a esta ciudad?
¿Qué vuelos hay disponibles para esta ciudad?
¿Qué vuelos hay disponibles para esta ciudad en esta fecha?
¿Qué vuelos hay disponibles para esta ciudad en esta fecha en esta clase?
¿Quiero viajar a esta ciudad?
Return:
Está herramienta te va a retornar un objeto con los topFlights y otherFlights. topFlights es un array con los vuelos más recomendados y otherFlights es un array con otros vuelos disponibles.
Muestrale la información al usuario de una manera clara y concisa.
"""
flight_suggests = {
    "name": "get_tickets_suggests",
    "description": msg_description,
    "parameters": {
        "type": "object",
        "properties": {
            "origin": {
                "type": "string",
                "description": "La ciudad de origen del vuelo",
            },
            "destination": {
                "type": "string",
                "description": "La ciudad de destino del vuelo",
            },
            "outbound_date": {
                "type": "string",
                "description": "La fecha de salida en formato YYYY-MM-DD",
            },
            "inbound_date": {
                "type": "string",
                "description": "La fecha de regreso en formato YYYY-MM-DD",
            },
            "travel_class": {
                "type": "string",
                "description": "La clase de viaje. Por defecto es ECONOMY, pero puede ser PREMIUM_ECONOMY, BUSINESS, FIRST.",
            },
        },
        "required": ["origin", "destination", "outbound_date", "inbound_date"],
        "additionalProperties": False
    }
}

In [9]:

tools = [{"type": "function", "function": flight_suggests}]

In [10]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    origin = arguments.get('origin')
    destination = arguments.get('destination')
    outbound_date = arguments.get('outbound_date')
    inbound_date = arguments.get('inbound_date')
    travel_class = arguments.get('travel_class', 'ECONOMY')
    itineraries = get_tickets_suggests(origin, destination, outbound_date, inbound_date, travel_class)
    response = {
        "role": "tool",
        "content": json.dumps({
            "origin": origin,
            "destination": destination,
            "outbound_date": outbound_date,
            "inbound_date": inbound_date,
            "travel_class": travel_class,
            "itineraries": itineraries
        }),
        "tool_call_id": message.tool_calls[0].id
    }
    return response

In [11]:
import tempfile
import subprocess
from io import BytesIO
from pydub import AudioSegment
import time

def play_audio(audio_segment):
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, "temp_audio.wav")
    try:
        audio_segment.export(temp_path, format="wav")
        time.sleep(3) # El estudiante Dominic consideró que esto era necesario. También podrías intentar comentar para ver si no es necesario en tu PC
        subprocess.call([
            "ffplay",
            "-nodisp",
            "-autoexit",
            "-hide_banner",
            temp_path
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    finally:
        try:
            os.remove(temp_path)
        except Exception:
            pass
 
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # Prueba también de reemplazar onyx por alloy
        input=message
    )
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play_audio(audio)

talker("Dime un chiste")

In [12]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    talker(reply)
    
    return history, image

In [13]:
# Código Gradio más complejo, ya que no estamos usando la interfaz de chat predeterminada.
# Si pasamos inbrowser=True en la última línea, aparecerá inmediatamente una ventana Gradio.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chatea con nuestro Agente de IA:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Buscando vuelos de LAX a JFK para el 2025-02-20 al 2025-02-23 en clase ECONOMY
{'status': True, 'message': 'Success', 'timestamp': 1739998881556, 'data': {'itineraries': {'topFlights': [{'departure_time': '20-02-2025 08:00 AM', 'arrival_time': '20-02-2025 04:26 PM', 'duration': {'raw': 326, 'text': '5 hr 26 min'}, 'flights': [{'departure_airport': {'airport_name': 'Aeropuerto Internacional de Los Ángeles', 'airport_code': 'LAX', 'time': '2025-2-20 08:00'}, 'arrival_airport': {'airport_name': 'Aeropuerto Internacional John F. Kennedy', 'airport_code': 'JFK', 'time': '2025-2-20 16:26'}, 'duration': {'raw': 326, 'text': '5 hr 26 min'}, 'airline': 'JetBlue', 'airline_logo': 'https://www.gstatic.com/flights/airline_logos/70px/B6.png', 'flight_number': 'B6 2824', 'aircraft': 'Airbus A320', 'seat': 'Above average legroom', 'legroom': '84 cm', 'extensions': ['Above average legroom (84 cm)', 'In-seat power & USB outlets', 'Live TV', 'Free Wi-Fi', 'Wi-Fi for a fee', 'Emissions estimate: 451 kg C